In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report,precision_recall_fscore_support
import warnings
import os

In [2]:
df1=pd.read_csv("case_study1.xlsx - case_study1.csv")
df2=pd.read_csv("case_study2.xlsx - case_study2.csv")

In [3]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51336 entries, 0 to 51335
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PROSPECTID            51336 non-null  int64  
 1   Total_TL              51336 non-null  int64  
 2   Tot_Closed_TL         51336 non-null  int64  
 3   Tot_Active_TL         51336 non-null  int64  
 4   Total_TL_opened_L6M   51336 non-null  int64  
 5   Tot_TL_closed_L6M     51336 non-null  int64  
 6   pct_tl_open_L6M       51336 non-null  float64
 7   pct_tl_closed_L6M     51336 non-null  float64
 8   pct_active_tl         51336 non-null  float64
 9   pct_closed_tl         51336 non-null  float64
 10  Total_TL_opened_L12M  51336 non-null  int64  
 11  Tot_TL_closed_L12M    51336 non-null  int64  
 12  pct_tl_open_L12M      51336 non-null  float64
 13  pct_tl_closed_L12M    51336 non-null  float64
 14  Tot_Missed_Pmnt       51336 non-null  int64  
 15  Auto_TL            

In [4]:
df1=df1.loc[df1['Age_Oldest_TL']!=-99999]  #removed the rows which has AGE_OLDEST_TL == -99999 since it was 40 rows out of 51000

In [5]:
df1.shape

(51296, 26)

In [6]:
columns_to_be_removed=[]
for i in df2.columns:
    if df2.loc[df2[i]==-99999].shape[0]>10000:
        columns_to_be_removed.append(i)

In [7]:
len(columns_to_be_removed)

8

In [8]:
df2.drop(columns_to_be_removed,axis=1,inplace=True)

In [9]:
df2.shape

(51336, 54)

In [10]:
for i in df2.columns:
    df2=df2.loc[df2[i]!=-99999]

In [11]:
df2.shape

(42066, 54)

In [12]:
for i in df2.columns:
    if i in df1.columns:
        print(i)

PROSPECTID


# Merging the two datasets

In [13]:
df=pd.merge(df1,df2,how='inner',left_on=['PROSPECTID'],right_on=['PROSPECTID'])

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 79 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PROSPECTID                  42064 non-null  int64  
 1   Total_TL                    42064 non-null  int64  
 2   Tot_Closed_TL               42064 non-null  int64  
 3   Tot_Active_TL               42064 non-null  int64  
 4   Total_TL_opened_L6M         42064 non-null  int64  
 5   Tot_TL_closed_L6M           42064 non-null  int64  
 6   pct_tl_open_L6M             42064 non-null  float64
 7   pct_tl_closed_L6M           42064 non-null  float64
 8   pct_active_tl               42064 non-null  float64
 9   pct_closed_tl               42064 non-null  float64
 10  Total_TL_opened_L12M        42064 non-null  int64  
 11  Tot_TL_closed_L12M          42064 non-null  int64  
 12  pct_tl_open_L12M            42064 non-null  float64
 13  pct_tl_closed_L12M          420

In [15]:
#check for the categorical columns
categorical_columns=[]
for i in df.columns:
    if df[i].dtype=='object':
        categorical_columns.append(i)

In [16]:
categorical_columns

['MARITALSTATUS',
 'EDUCATION',
 'GENDER',
 'last_prod_enq2',
 'first_prod_enq2',
 'Approved_Flag']

In [17]:
for i in categorical_columns:
    print(df[i].value_counts())
    print("\n")

MARITALSTATUS
Married    30886
Single     11178
Name: count, dtype: int64


EDUCATION
GRADUATE          14140
12TH              11703
SSC                7241
UNDER GRADUATE     4572
OTHERS             2291
POST-GRADUATE      1898
PROFESSIONAL        219
Name: count, dtype: int64


GENDER
M    37345
F     4719
Name: count, dtype: int64


last_prod_enq2
ConsumerLoan    16480
others          13653
PL               7553
CC               2195
AL               1353
HL                830
Name: count, dtype: int64


first_prod_enq2
others          20640
ConsumerLoan    11075
PL               4431
AL               2641
CC               1988
HL               1289
Name: count, dtype: int64


Approved_Flag
P2    25452
P3     6440
P4     5264
P1     4908
Name: count, dtype: int64




In [18]:
for i in categorical_columns:
    chi2,pval,_,_ = chi2_contingency(pd.crosstab(df[i],df['Approved_Flag']))
    print(i,'--->',pval)
#since all the five features have pval<=0.05, we will accept all these features

MARITALSTATUS ---> 3.578180861038862e-233
EDUCATION ---> 2.6942265249737532e-30
GENDER ---> 1.907936100186563e-05
last_prod_enq2 ---> 0.0
first_prod_enq2 ---> 7.84997610555419e-287
Approved_Flag ---> 0.0


In [19]:
numeric_columns=[]
for i in df.columns:
    if df[i].dtype!='object' and i not in ['PROSPECTID','Approved_Flag']:
        numeric_columns.append(i)

In [20]:
len(numeric_columns)

72

In [21]:
#VIF Sequentially Check
vif_data=df[numeric_columns]
total_columns=vif_data.shape[1]
columns_to_be_kept=[]
column_index=0

for i in range(0,total_columns):
    vif_value=variance_inflation_factor(vif_data,column_index)
    print(column_index,'--->',vif_value)
    
    if vif_value<=6:
        columns_to_be_kept.append(numeric_columns[i])
        column_index=column_index+1
    else:
        vif_data=vif_data.drop([numeric_columns[i]],axis=1)

C:\Users\HP\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 ---> inf


C:\Users\HP\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 ---> inf
0 ---> 11.320180023967996
0 ---> 8.363698035000336
0 ---> 6.520647877790928
0 ---> 5.149501618212625
1 ---> 2.611111040579735


C:\Users\HP\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


2 ---> inf
2 ---> 1788.7926256209232
2 ---> 8.601028256477228
2 ---> 3.8328007921530785
3 ---> 6.0996533816467355
3 ---> 5.5813520096427585
4 ---> 1.985584353098778


C:\Users\HP\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


5 ---> inf
5 ---> 4.809538302819343
6 ---> 23.270628983464636
6 ---> 30.595522588100053
6 ---> 4.3843464059655854
7 ---> 3.0646584155234238
8 ---> 2.898639771299252
9 ---> 4.377876915347322
10 ---> 2.2078535836958433
11 ---> 4.916914200506864
12 ---> 5.214702030064725
13 ---> 3.3861625024231476
14 ---> 7.840583309478997
14 ---> 5.255034641721438


C:\Users\HP\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


15 ---> inf
15 ---> 7.380634506427232
15 ---> 1.421005001517573
16 ---> 8.083255010190323
16 ---> 1.6241227524040112
17 ---> 7.257811920140003
17 ---> 15.59624383268298
17 ---> 1.8258570471324314
18 ---> 1.5080839450032664
19 ---> 2.172088834824577
20 ---> 2.623397553527229
21 ---> 2.2959970812106167
22 ---> 7.360578319196446
22 ---> 2.1602387773102554
23 ---> 2.8686288267891475
24 ---> 6.458218003637272
24 ---> 2.8474118865638256
25 ---> 4.753198156284083
26 ---> 16.22735475594825
26 ---> 6.424377256363877
26 ---> 8.887080381808687
26 ---> 2.3804746142952653
27 ---> 8.609513476514548
27 ---> 13.06755093547673
27 ---> 3.5000400566546555
28 ---> 1.9087955874813773
29 ---> 17.006562234161628
29 ---> 10.730485153719197
29 ---> 2.3538497522950275
30 ---> 22.104855915136433
30 ---> 2.7971639638512924
31 ---> 3.424171203217697
32 ---> 10.175021454450935
32 ---> 6.408710354561301
32 ---> 1.0011511962625612
33 ---> 3.069197305397274
34 ---> 2.8091261600643724
35 ---> 20.249538381980678
35 --->

In [22]:
vif_data.shape

(42064, 39)

In [23]:
from scipy.stats import f_oneway
columns_to_be_kept_numerical=[]

for i in columns_to_be_kept:
    a=list(df[i])
    b=list(df['Approved_Flag'])
    
    group_P1=[value for value,group in zip(a,b) if group=='P1']
    group_P2=[value for value,group in zip(a,b) if group=='P2']
    group_P3=[value for value,group in zip(a,b) if group=='P3']
    group_P4=[value for value,group in zip(a,b) if group=='P4']
    
    f_statistic,p_value=f_oneway(group_P1,group_P2,group_P3,group_P4)
    
    if p_value<=0.5:
        columns_to_be_kept_numerical.append(i)

In [24]:
#done with feature engineering part

In [25]:
categorical_columns

['MARITALSTATUS',
 'EDUCATION',
 'GENDER',
 'last_prod_enq2',
 'first_prod_enq2',
 'Approved_Flag']

In [26]:
# GRADUATE          14140
# 12TH              11703
# SSC                7241
# UNDER GRADUATE     4572
# OTHERS             2291
# POST-GRADUATE      1898
# PROFESSIONAL        219

In [27]:
df.loc[df['EDUCATION']=='SSC',['EDUCATION']]=1
df.loc[df['EDUCATION']=='12TH',['EDUCATION']]=2
df.loc[df['EDUCATION']=='UNDER GRADUATE',['EDUCATION']]=3
df.loc[df['EDUCATION']=='GRADUATE',['EDUCATION']]=3
df.loc[df['EDUCATION']=='OTHERS',['EDUCATION']]=1
df.loc[df['EDUCATION']=='POST-GRADUATE',['EDUCATION']]=4
df.loc[df['EDUCATION']=='PROFESSIONAL',['EDUCATION']]=3

In [28]:
df['EDUCATION'].value_counts()

EDUCATION
3    18931
2    11703
1     9532
4     1898
Name: count, dtype: int64

In [29]:
df['EDUCATION']=df['EDUCATION'].astype(int)

In [30]:
df_encoded=pd.get_dummies(df,columns=['MARITALSTATUS','GENDER','last_prod_enq2','first_prod_enq2'])

In [32]:
df_encoded.columns

Index(['PROSPECTID', 'Total_TL', 'Tot_Closed_TL', 'Tot_Active_TL',
       'Total_TL_opened_L6M', 'Tot_TL_closed_L6M', 'pct_tl_open_L6M',
       'pct_tl_closed_L6M', 'pct_active_tl', 'pct_closed_tl',
       'Total_TL_opened_L12M', 'Tot_TL_closed_L12M', 'pct_tl_open_L12M',
       'pct_tl_closed_L12M', 'Tot_Missed_Pmnt', 'Auto_TL', 'CC_TL',
       'Consumer_TL', 'Gold_TL', 'Home_TL', 'PL_TL', 'Secured_TL',
       'Unsecured_TL', 'Other_TL', 'Age_Oldest_TL', 'Age_Newest_TL',
       'time_since_recent_payment', 'num_times_delinquent',
       'max_recent_level_of_deliq', 'num_deliq_6mts', 'num_deliq_12mts',
       'num_deliq_6_12mts', 'num_times_30p_dpd', 'num_times_60p_dpd',
       'num_std', 'num_std_6mts', 'num_std_12mts', 'num_sub', 'num_sub_6mts',
       'num_sub_12mts', 'num_dbt', 'num_dbt_6mts', 'num_dbt_12mts', 'num_lss',
       'num_lss_6mts', 'num_lss_12mts', 'recent_level_of_deliq', 'tot_enq',
       'CC_enq', 'CC_enq_L6m', 'CC_enq_L12m', 'PL_enq', 'PL_enq_L6m',
       'PL_enq_L12

In [33]:
df_encoded['MARITALSTATUS_Married']=df_encoded['MARITALSTATUS_Married'].astype(int)
df_encoded['MARITALSTATUS_Single']=df_encoded['MARITALSTATUS_Single'].astype(int)
df_encoded['GENDER_F']=df_encoded['GENDER_F'].astype(int)
df_encoded['GENDER_M']=df_encoded['GENDER_M'].astype(int)
df_encoded['last_prod_enq2_AL']=df_encoded['last_prod_enq2_AL'].astype(int)
df_encoded['last_prod_enq2_CC']=df_encoded['last_prod_enq2_CC'].astype(int)
df_encoded['last_prod_enq2_ConsumerLoan']=df_encoded['last_prod_enq2_ConsumerLoan'].astype(int)
df_encoded['last_prod_enq2_HL']=df_encoded['last_prod_enq2_HL'].astype(int)
df_encoded['last_prod_enq2_PL']=df_encoded['last_prod_enq2_PL'].astype(int)
df_encoded['last_prod_enq2_others']=df_encoded['last_prod_enq2_others'].astype(int)
df_encoded['first_prod_enq2_AL']=df_encoded['first_prod_enq2_AL'].astype(int)
df_encoded['first_prod_enq2_CC']=df_encoded['first_prod_enq2_CC'].astype(int)
df_encoded['first_prod_enq2_ConsumerLoan']=df_encoded['first_prod_enq2_ConsumerLoan'].astype(int)
df_encoded['first_prod_enq2_HL']=df_encoded['first_prod_enq2_HL'].astype(int)
df_encoded['first_prod_enq2_PL']=df_encoded['first_prod_enq2_PL'].astype(int)
df_encoded['first_prod_enq2_others']=df_encoded['first_prod_enq2_others'].astype(int)

In [51]:
df_encoded.head() #now True And False Converted to 0s and 1s

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,last_prod_enq2_ConsumerLoan,last_prod_enq2_HL,last_prod_enq2_PL,last_prod_enq2_others,first_prod_enq2_AL,first_prod_enq2_CC,first_prod_enq2_ConsumerLoan,first_prod_enq2_HL,first_prod_enq2_PL,first_prod_enq2_others
0,1,5,4,1,0,0,0.000,0.0,0.200,0.800,...,0,0,1,0,0,0,0,0,1,0
1,2,1,0,1,0,0,0.000,0.0,1.000,0.000,...,1,0,0,0,0,0,1,0,0,0
2,3,8,0,8,1,0,0.125,0.0,1.000,0.000,...,1,0,0,0,0,0,0,0,0,1
3,5,3,2,1,0,0,0.000,0.0,0.333,0.667,...,0,0,0,0,1,0,0,0,0,0
4,6,6,5,1,0,0,0.000,0.0,0.167,0.833,...,1,0,0,0,0,0,0,0,1,0


In [52]:
df_encoded['Approved_Flag'].value_counts()

Approved_Flag
P2    25452
P3     6440
P4     5264
P1     4908
Name: count, dtype: int64

In [36]:
y=df_encoded['Approved_Flag']
X=df_encoded.drop(['Approved_Flag'],axis=1)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

# 1. Random Forest Model

In [37]:
rf=RandomForestClassifier(n_estimators=200,random_state=42)
rf.fit(X_train,y_train)
y_pred=rf.predict(X_test)

In [40]:
accuracy=accuracy_score(y_test,y_pred)
print('Accuracy: ',accuracy)
precision,recall,f1_score,_=precision_recall_fscore_support(y_test,y_pred)

for i,v in enumerate (['p1','p2','p3','p4']):
    print(f"class={v}")
    print()
    print(f"Precision: {precision[i]}")
    print(f"Recall:{recall[i]}")
    print(f"F1_score: {f1_score[i]}")
    print("\n")

Accuracy:  0.9900154522762391
class=p1

Precision: 0.9465290806754222
Recall:0.995069033530572
F1_score: 0.9701923076923077


class=p2

Precision: 0.9954617205998422
Recall:1.0
F1_score: 0.9977256995945812


class=p3

Precision: 0.9968102073365231
Recall:0.9433962264150944
F1_score: 0.9693679720822024


class=p4

Precision: 1.0
Recall:0.9961127308066083
F1_score: 0.9980525803310614




# 2. XGBoost Model

In [44]:
!pip install xgboost

   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB 1.3 MB/s eta 0:01:18
   ---------------------------------------- 0.2/99.8 MB 5.0 MB/s eta 0:00:20
    --------------------------------------- 1.5/99.8 MB 13.6 MB/s eta 0:00:08
    --------------------------------------- 1.8/99.8 MB 11.4 MB/s eta 0:00:09
    --------------------------------------- 2.1/99.8 MB 11.4 MB/s eta 0:00:09
    --------------------------------------- 2.5/99.8 MB 10.5 MB/s eta 0:00:10
   - -------------------------------------- 2.7/99.8 MB 9.9 MB/s eta 0:00:10
   - -------------------------------------- 2.9/99.8 MB 8.9 MB/s eta 0:00:11
   - -------------------------------------- 3.2/99.8 MB 8.9 MB/s eta 0:00:11
   - -------------------------------------- 3.5/99.8 MB 8.5 MB/s eta 0:00:12
   - -------------------------------------- 3.7/99.8 MB 8.1 MB/s eta 0:00:12
   - -------------------------------------- 3.9/99.8 MB 8.0 MB/s eta 0:00:12
  

In [48]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

xgb_classifier=xgb.XGBClassifier(objective='multi:softmax',num_class=4)
label_encoder=LabelEncoder()

y=df_encoded['Approved_Flag']
X=df_encoded.drop(['Approved_Flag'],axis=1)

y_encoded=label_encoder.fit_transform(y)
X_train,X_test,y_train,y_test=train_test_split(X,y_encoded,test_size=0.2,random_state=42)

xgb_classifier.fit(X_train,y_train)
y_pred=xgb_classifier.predict(X_test)
accuracy=accuracy_score(y_test,y_pred)
print(f"Accuracy: {accuracy}")
precision,recall,f1_score,_=precision_recall_fscore_support(y_test,y_pred)

for i,v in enumerate (['p1','p2','p3','p4']):
    print(f"class={v}")
    print()
    print(f"Precision: {precision[i]}")
    print(f"Recall:{recall[i]}")
    print(f"F1_score: {f1_score[i]}")
    print("\n")

Accuracy: 1.0
class=p1

Precision: 1.0
Recall:1.0
F1_score: 1.0


class=p2

Precision: 1.0
Recall:1.0
F1_score: 1.0


class=p3

Precision: 1.0
Recall:1.0
F1_score: 1.0


class=p4

Precision: 1.0
Recall:1.0
F1_score: 1.0




# 3. Decision Tree Classifier

In [49]:
from sklearn.tree import DecisionTreeClassifier
y=df_encoded['Approved_Flag']
X=df_encoded.drop(['Approved_Flag'],axis=1)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
dt=DecisionTreeClassifier(max_depth=20,min_samples_split=10)
dt.fit(X_train,y_train)
y_pred=dt.predict(X_test)
accuracy=accuracy_score(y_test,y_pred)
print('Accuracy: ',accuracy)
precision,recall,f1_score,_=precision_recall_fscore_support(y_test,y_pred)

for i,v in enumerate (['p1','p2','p3','p4']):
    print(f"class={v}")
    print()
    print(f"Precision: {precision[i]}")
    print(f"Recall:{recall[i]}")
    print(f"F1_score: {f1_score[i]}")
    print("\n")

Accuracy:  1.0
class=p1

Precision: 1.0
Recall:1.0
F1_score: 1.0


class=p2

Precision: 1.0
Recall:1.0
F1_score: 1.0


class=p3

Precision: 1.0
Recall:1.0
F1_score: 1.0


class=p4

Precision: 1.0
Recall:1.0
F1_score: 1.0


